In [167]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import re
from sklearn.feature_extraction.text import CountVectorizer

In [168]:
all_verbs = pd.read_csv('./all_verbs_tree.csv').reset_index(drop = True)
all_verbs

,Unnamed: 0,Unnamed: 0.1,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33867,33867,33867,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
33868,33868,33868,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
33869,33869,33869,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000,0,n,8,5
33870,33870,33870,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000,0,n,8,5


In [169]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [170]:
all_characters = all_verbs['Generated Verb-SR'].sum() #concatinating all characters together
all_characters[:200]

'amneˈgerijemɛamneˈgeriiamneˈgerijeamneˈgeriijɛamneˈgerijomɛamneˈgerijʌamneˈgerijendɛangerijeˈmɛangeriiˈangerijeˈangeriiˈjɛangerijoˈmɛangerijʌˈangerijeˈndɛaneˈgerijemɛaneˈgeriianeˈgerijeaneˈgeriijɛaneˈ'

In [171]:
all_characters = sorted(list(set([char for char in all_characters]))) #set for unique characters # list for making a list out of those characters
all_characters

['*',
 ':',
 'a',
 'b',
 'd',
 'e',
 'f',
 'g',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'z',
 'ø',
 'ɛ',
 'ɢ',
 'ʃ',
 'ʌ',
 'ʒ',
 'ˈ',
 '͡',
 'χ']

In [172]:
char_index = {char: idx+1 for idx, char in enumerate(all_characters) } # input cha, gives you a no #inumerate: iterate a list, returns the element and its index
index_char = {idx+1: char for idx, char in enumerate(all_characters)}
char_index
 # plus one is to start from 1

{'*': 1,
 ':': 2,
 'a': 3,
 'b': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'z': 22,
 'ø': 23,
 'ɛ': 24,
 'ɢ': 25,
 'ʃ': 26,
 'ʌ': 27,
 'ʒ': 28,
 'ˈ': 29,
 '͡': 30,
 'χ': 31}

In [173]:
index_char

{1: '*',
 2: ':',
 3: 'a',
 4: 'b',
 5: 'd',
 6: 'e',
 7: 'f',
 8: 'g',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'r',
 18: 's',
 19: 't',
 20: 'u',
 21: 'v',
 22: 'z',
 23: 'ø',
 24: 'ɛ',
 25: 'ɢ',
 26: 'ʃ',
 27: 'ʌ',
 28: 'ʒ',
 29: 'ˈ',
 30: '͡',
 31: 'χ'}

In [174]:
# next is for each word, you need to produce the list that has the indexes of its chas

In [175]:
all_verbs_list = all_verbs['Generated Verb-SR'].tolist()
print(all_verbs_list[:100])

X = [[char_index[char] for char in verb] for verb in all_verbs_list] #predictor variable

['amneˈgerijemɛ', 'amneˈgerii', 'amneˈgerije', 'amneˈgeriijɛ', 'amneˈgerijomɛ', 'amneˈgerijʌ', 'amneˈgerijendɛ', 'angerijeˈmɛ', 'angeriiˈ', 'angerijeˈ', 'angeriiˈjɛ', 'angerijoˈmɛ', 'angerijʌˈ', 'angerijeˈndɛ', 'aneˈgerijemɛ', 'aneˈgerii', 'aneˈgerije', 'aneˈgeriijɛ', 'aneˈgerijomɛ', 'aneˈgerijʌ', 'aneˈgerijendɛ', 'aˈgerijemɛ', 'aˈgerii', 'aˈgerije', 'aˈgeriijɛ', 'aˈgerijomɛ', 'aˈgerijʌ', 'aˈgerijendɛ', 'amneˈgerijɛstimɛ', 'amneˈgerijɛstiʃɛ', 'amneˈgerijast', 'amneˈgerijastɛ', 'amneˈgerijɛstimon', 'amneˈgerijɛstijon', 'amneˈgerijɛstindɛ', 'angerijɛˈstimɛ', 'angerijɛˈstiʃɛ', 'angerijaˈst', 'angerijaˈstɛ', 'angerijɛˈstimon', 'angerijɛˈstijon', 'angerijɛˈstindɛ', 'aneˈgerijɛstimɛ', 'aneˈgerijɛstiʃɛ', 'aneˈgerijast', 'aneˈgerijastɛ', 'aneˈgerijɛstimon', 'aneˈgerijɛstijon', 'aneˈgerijɛstindɛ', 'agerijɛˈstimɛ', 'agerijɛˈstiʃɛ', 'agerijaˈst', 'agerijaˈstɛ', 'agerijɛˈstimon', 'agerijɛˈstijon', 'agerijɛˈstindɛ', 'aneˈgerijɛstɛjmɛ', 'aneˈgerijɛstɛjʃɛ', 'aneˈgerijɛsti', 'aneˈgerijɛstijʌ', 'aneˈge

In [176]:
from tensorflow.keras.utils import pad_sequences


In [177]:
max_len = max(len(item) for item in X)
max_len

25

In [178]:
X = pad_sequences(X, max_len) #tags zeros to the front

In [179]:
features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
y = all_verbs[features]
y

,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity
0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans
1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans
2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans
3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans
4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans
...,...,...,...,...,...,...,...,...
33867,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans
33868,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans
33869,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans
33870,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans


In [180]:
X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X, y, test_size = 0.33, random_state = 1)

In [226]:
feature = 'Voice'
le = LabelEncoder()
y_train = le.fit_transform(y_train_pre_encoded[feature]) #le.fit() and le.transform() together make le.fit_transform()
y_test = le.transform(y_test_pre_encoded[feature])
print(le.classes_)

['active' 'passive']


In [227]:
lstm_model = tf.keras.models.Sequential()

lstm_model.add(tf.keras.layers.Embedding(input_dim = len(char_index)+1, output_dim = 32, mask_zero = True)) # True ignores zeros # all sequences needs to have the same length but zero willl be ignored
lstm_model.add(tf.keras.layers.LSTM(32)) #can change later
lstm_model.add(tf.keras.layers.Dense(len(le.classes_), activation = 'softmax')) #one output per category

lstm_model.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy') #cat cross can have more than two target classes. Sparse : bc we are giving it an integer label

In [228]:
lstm_model.fit(X_train, y_train, epochs = 10) # passes the df 5 times and train on them

Epoch 1/10
710/710 [==============================] - 16s 15ms/step - loss: 0.1260
Epoch 2/10
710/710 [==============================] - 12s 17ms/step - loss: 0.0145
Epoch 3/10
710/710 [==============================] - 11s 16ms/step - loss: 0.0065
Epoch 4/10
710/710 [==============================] - 13s 18ms/step - loss: 0.0038
Epoch 5/10
710/710 [==============================] - 11s 16ms/step - loss: 0.0020
Epoch 6/10
710/710 [==============================] - 12s 17ms/step - loss: 0.0022
Epoch 7/10
710/710 [==============================] - 11s 16ms/step - loss: 0.0012
Epoch 8/10
710/710 [==============================] - 11s 16ms/step - loss: 0.0017
Epoch 9/10
710/710 [==============================] - 12s 17ms/step - loss: 0.0016
Epoch 10/10
710/710 [==============================] - 11s 16ms/step - loss: 7.8481e-04


In [229]:
predictions = lstm_model.predict(X_test) #predicted probability

350/350 [==============================] - 4s 4ms/step


In [230]:
y_pred = np.argmax(predictions, axis = 1) # translates from probabilies to classes

In [231]:
accuracy_score(y_test, y_pred)

0.9997316156736447

In [232]:
confusion_matrix(y_test, y_pred)

array([[6297,    3],
       [   0, 4878]])

Modality: 99
array([[ 218,    1,   27],
       [   0, 7816,    3],
       [  35,    8, 3070]])

Imperative, Indicative, Subjunctive
Aspect: 98
array([[3115,    3,    0,    0],
       [  15, 3321,   56,    0],
       [   0,  118, 2975,    0],
       [   0,    0,    0, 1575]])
['imperfective' 'neutral' 'perfect' 'pluperfect']

Modality:99
array([[ 211,    0,   35],
       [   2, 7809,    8],
       [  30,    6, 3077]])
['imperative' 'indicative' 'subjunctive']



Causation:1
array([[3527,    0],
       [   0, 7651]])
['causative' 'non-causative']

Person: 95
array([[1619,    0,    0,    0,    0,    0,    0],
       [   0, 1565,    0,    0,    0,    0,    0],
       [   0,    0, 1623,    0,    0,    0,    0],
       [   1,    0,    1, 1703,    0,    3,    5],
       [   0,    0,    0,    0, 1520,    0,    0],
       [   0,    0,    0,    0,    0, 1103,  441],
       [   0,    0,    0,    2,    0,   22, 1570]])
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']


Transitivity: 98
array([[6080,  106],
       [  70, 4922]])
       ['intrans' 'trans']


Voice: 99
array([[6297,    3],
       [   0, 4878]])

['active' 'passive']


def LSTM_models(feature):
    le = LabelEncoder()
    y_train = le.fit_transform(y_train_pre_encoded[feature]) #le.fit() and le.transform() together make le.fit_transform()
    y_test = le.transform(y_test_pre_encoded[feature])
    lstm_model = tf.keras.models.Sequential()

    lstm_model.add(tf.keras.layers.Embedding(input_dim = len(char_index)+1, output_dim = 32, mask_zero = True)) # True ignores zeros # all sequences needs to have the same length but zero willl be ignored
    lstm_model.add(tf.keras.layers.LSTM(32)) #can change later
    lstm_model.add(tf.keras.layers.Dense(len(le.classes_), activation = 'softmax')) #one output per category

    lstm_model.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy') #cat cross can have more than two target classes. Sparse : bc we are giving it an integer label
    lstm_model.fit(X_train, y_train, epochs = 10) # passes the df 5 times and train on them
    predictions = lstm_model.predict(X_test) #predicted probability
    y_pred = np.argmax(predictions, axis = 1) # translates from probabilies to classes
    return le.classes_, accuracy_score(y_test, y_pred), confusion_matrix(y_test, y_pred)

features = ['Tense', 'Aspect', 'Modality', 'Person', 'Voice', 'Polarity', 'Transitivity']
for feature in features:
    print(f'FEATURE: {feature}')
    le_classes, accuracy_score, confusion_matrix = LSTM_models(feature)
    print(f'CLASSES: {le_classes}')
    print(f'ACCURACY_SCORE: {accuracy_score}')
    print(f'CONFUSION_MATRIX: ')
    print(confusion_matrix)
    